In [6]:
import os
import pickle
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import RepeatVector, Permute
from keras.layers import Multiply, Lambda
import keras.backend as K 
from keras.models import Model
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-06-03 22:31:22.582613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 22:31:25.245360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-06-03 22:31:31.378223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 22:31:31.802538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 22:31:31.802745: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


### Load data from the Oskar Kolberg's Dataset
https://webesac.pcss.pl/

In [7]:
data = []

In [8]:
import pickle
files = ["data_kolberg"]
for file in files:
    with open("../data/Oskar Kolberg's Dataset/{}.pkl".format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

In [9]:
number_of_pieces = len(data)
number_of_pieces

19092

In [6]:
### Remove too long or too short pieces

In [10]:
pieces_lengths = dict()
for piece in data:
    if len(piece[0]) in pieces_lengths.keys():
        pieces_lengths[len(piece[0])] += 1
    else:
        pieces_lengths[len(piece[0])] = 1

pieces_lengths = sorted(pieces_lengths.items(), key=lambda x: x[1], reverse=True)
pieces_lengths = [ (length, count) for length, count in pieces_lengths if count > 0.02 * number_of_pieces]

In [11]:
pieces_lengths

[(32, 1089),
 (28, 981),
 (36, 714),
 (34, 693),
 (30, 650),
 (26, 639),
 (24, 600),
 (29, 599),
 (33, 573),
 (38, 559),
 (31, 558),
 (40, 553),
 (37, 524),
 (35, 513),
 (27, 478),
 (42, 460),
 (39, 457),
 (41, 411)]

In [12]:
pieces_lengths = [ length for length, count in pieces_lengths ]

In [13]:
lower_bound = min(pieces_lengths)
upper_bound = max(pieces_lengths)

In [14]:
# The length of the shortest piece
lower_bound

24

In [15]:
# The length of the longest piece
upper_bound

42

In [16]:
cleaned_data = []
for piece in data:
    if len(piece[0]) in pieces_lengths:
        cleaned_data.append(piece)

In [17]:
data = cleaned_data
number_of_pieces = len(data)
number_of_pieces

11051

### Remove too long or too short phrases

Indicate the range of the most common lengths of phrases

In [18]:
notes_data = []
bars_data = [] 
phrases_data = []

for piece in data:
    notes_data.extend(piece[0])
    bars_data.extend(piece[2])
    phrases_data.extend(piece[3])

In [19]:
print(data[0][0])
print(data[0][2])
print(data[0][3])

[60, 64, 67, 67, 67, 70, 67, 64, 65, 69, 65, 62, 60, 64, 67, 60, 60, 64, 67, 67, 67, 71, 72, 67, 67, 69, 65, 62, 64, 60]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [20]:
phrases_lengths = dict()
counter = 0
for i in range(len(phrases_data)):
    counter+=1
    if phrases_data[i] == 1:
        if counter in phrases_lengths.keys():
            phrases_lengths[counter] += 1
        else:
            phrases_lengths[counter] = 1
        counter = 0

phrases_lengths = sorted(phrases_lengths.items(), key=lambda x: x[1], reverse=True)
phrases_lengths = [ (length, count) for length, count in phrases_lengths if count > 0.1 * number_of_pieces]
phrases_lengths

[(14, 3159),
 (8, 2811),
 (16, 2551),
 (12, 2328),
 (13, 1993),
 (15, 1846),
 (10, 1836),
 (9, 1781),
 (7, 1618),
 (18, 1458),
 (17, 1386),
 (11, 1126)]

In [21]:
phrases_lengths = [ length for length, count in phrases_lengths ]

Remove pieces with irregular phrases

In [22]:
seq_len = phrases_lengths[0]
lower_bound = min(phrases_lengths)
upper_bound = max(phrases_lengths)

In [23]:
# The most common phrases length ==> the lenght of a sequence
seq_len = 36
seq_len

36

In [24]:
# The shortest phrase lenght we consider
lower_bound

7

In [25]:
# The longest phrase lenght we consider
upper_bound

18

In [26]:
pieces = []

for piece in data:
    current_phrase_length = 0
    skip = False

    if len(piece[0]) < upper_bound:
        skip = True
    else:
        for i, digit in enumerate(piece[3]):
            current_phrase_length += 1
            if digit == 1:
                # Skip pieces with irregular phrases
                if current_phrase_length not in phrases_lengths:
                    skip = True
                    break
                # Skip pieces with phrases that don't end at the end of a measure
                if piece[2][i] != 1:
                    skip = True
                    break
                current_phrase_length = 0
    if skip:
        continue
    else:
        pieces.append(piece)

In [27]:
number_of_pieces = len(pieces)
number_of_pieces

6857

In [28]:
cleaned_data = pieces
# save the train and test data to pickle files
with open("../data/Oskar Kolberg's Dataset/data_kolberg_cleaned.pkl", 'wb') as handle:
    pickle.dump(cleaned_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Split the data into train and test data

In [25]:
# Train test split
pieces_train, pieces_test = train_test_split(pieces, test_size=0.2, random_state=42)

In [26]:
len(pieces_train), len(pieces_test)

(5485, 1372)

In [27]:
# save the train and test data to pickle files
with open("../data/Oskar Kolberg's Dataset/train/pieces_train.pkl", 'wb') as handle:
    pickle.dump(pieces_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("../data/Oskar Kolberg's Dataset/test/pieces_test.pkl", 'wb') as handle:
    pickle.dump(pieces_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Extract train data

In [28]:
notes_input = []
bars_input = []
phrases_input = []

for piece in pieces_train:
    notes = piece[0]
    notes_input.extend([129 for i in range(seq_len)])
    notes_input.extend(notes)

    bars = piece[2]
    bars_input.extend([0 for i in range(seq_len)])
    bars_input.extend(bars)
    
    phrases = piece[3]
    phrases_input.extend([0 for i in range(seq_len)])
    phrases_input.extend(phrases)

In [29]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the bars_input: {}".format(len(bars_input)))
print("Twenty first elements: {}".format(bars_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

Length of the notes_input: 369729
Twenty first elements: [129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129]
Length of the bars_input: 369729
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Length of the phrases_input: 369729
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Prepare data for embedding

Map durations and notes to integers

In [30]:
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
bars_to_int = {}
int_to_bars = {}

for index, note in enumerate(sorted(set(notes_data).union({129}))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_data))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

for index, bar in enumerate(sorted(set(bars_data))):
    bars_to_int[bar] = index
    int_to_bars[index] = bar

notes_network_input = [ notes_to_int[note] for note in notes_input ]
bars_network_input = [ bars_to_int[bar] for bar in bars_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [31]:
# Before
print(sorted(set(notes_input)))

[43, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 89, 128, 129]


In [32]:
# After
print(sorted(set(notes_network_input)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41]


In [33]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the bars_input: {}".format(len(bars_network_input)))
print("Twenty first elements: {}".format(bars_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

Length of the notes_input: 369729
Twenty first elements: [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41]
Length of the bars_input: 369729
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Length of the phrases_input: 369729
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
n_notes = len(notes_to_int)
n_phrases = len(phrases_to_int)
n_bars = len(bars_to_int)

In [35]:
print("Number of distinct notes: {}".format(n_notes))

Number of distinct notes: 42


In [36]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases, "bars_to_int" : bars_to_int, "int_to_bars" : int_to_bars }
    # A new file will be created
    pickle.dump(dictionary, file)

### Prepare network input and output

In [37]:
def prepare_sequences(notes, bars, phrases, seq_len = 32):

    notes_input = []
    bars_input = []
    phrases_output = []

    for i in range(len(notes) - seq_len):
        notes_input.append(notes[i:i + seq_len])
        bars_input.append(bars[i:i + seq_len])

        phrases_output.append(phrases[i + seq_len])

    n_patterns = len(notes_input)

    notes_input = np.reshape(notes_input, (n_patterns, seq_len))
    bars_input = np.reshape(bars_input, (n_patterns, seq_len))
    network_input = [notes_input, bars_input]

    phrases_output = np_utils.to_categorical(phrases_output, num_classes=2)
    network_output = [phrases_output]

    return (network_input, network_output)

In [38]:
network_input, network_output = prepare_sequences(notes_network_input, bars_network_input, phrases_network_input, seq_len)

In [39]:
print('note input')
print(network_input[0][0])
print('bars input')
print(network_input[1][0])
print('phrases_output')
print(network_output[0][0])

note input
[41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41]
bars input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
phrases_output
[1. 0.]


### Create neural network

In [40]:
def create_network(n_notes, n_bars, n_phrases, embed_size = 100, rnn_units = 256, use_attention = False):

    notes_in = Input(shape = (None,))
    bars_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x3 = Embedding(n_bars, embed_size)(bars_in)

    x = Concatenate()([x1,x3])

    x = LSTM(rnn_units, return_sequences=True)(x)

    if use_attention:

        x = LSTM(rnn_units, return_sequences=True)(x)

        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)
        alpha = Activation('softmax')(e)

        alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
    else:
        c = LSTM(rnn_units)(x)
                                    
    phrases_out = Dense(n_phrases, activation = 'softmax', name = 'phrase')(c)
   
    model = Model([notes_in, bars_in], [phrases_out])
    

    if use_attention:
        att_model = Model([notes_in, bars_in], alpha)
    else:
        att_model = None


    opti = RMSprop(lr = 0.001)
    model.compile(loss=['binary_crossentropy', 'binary_crossentropy'], optimizer=opti)

    return model, att_model

In [41]:
embed_size = 42
rnn_units = 256
use_attention = True
model, att_model = create_network(n_notes, n_bars, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

2023-05-29 01:35:10.096468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 01:35:10.098734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 01:35:10.100232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 42)     1764        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 42)     84          ['input_2[0][0]']                
                                                                                              

2023-05-29 01:35:10.347743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 01:35:10.349942: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 01:35:10.351454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

### Train the model

In [42]:
weights_folder = "weights"

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=30
          , batch_size=100
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

Epoch 1/30


2023-05-29 01:35:10.923447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 01:35:10.925956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 01:35:10.927751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2958/2958 [==============================] - ETA: 0s - loss: 0.1290

2023-05-29 01:43:04.697218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 01:43:04.699195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 01:43:04.700655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2958/2958 [==============================] - 520s 174ms/step - loss: 0.1290 - val_loss: 0.1204
Epoch 2/30
2958/2958 [==============================] - 527s 178ms/step - loss: 0.1099 - val_loss: 0.1006
Epoch 3/30
2958/2958 [==============================] - 536s 181ms/step - loss: 0.0965 - val_loss: 0.0936
Epoch 4/30
2958/2958 [==============================] - 537s 182ms/step - loss: 0.0830 - val_loss: 0.0790
Epoch 5/30
2958/2958 [==============================] - 534s 180ms/step - loss: 0.0770 - val_loss: 0.0794
Epoch 6/30
2958/2958 [==============================] - 525s 177ms/step - loss: 0.0744 - val_loss: 0.0738
Epoch 7/30
2958/2958 [==============================] - 525s 178ms/step - loss: 0.0719 - val_loss: 0.0737
Epoch 8/30
2958/2958 [==============================] - 529s 179ms/step - loss: 0.0703 - val_loss: 0.0713
Epoch 9/30
2958/2958 [==============================] - 530s 179ms/step - loss: 0.0688 - val_loss: 0.0704
Epoch 10/30
2958/2958 [==============================] - 